In [134]:
import numpy as np
from IPython.display import display

np.set_printoptions(suppress=True)

#np.zeros(5)
sum([1,2,3,4])
display('{} {}'.format('one', 'two'))

'one two'

In [135]:
class HMM:
    def __init__(self, hidden_states, visible_symbols):
        
        # first hidden state is actually w0 (omega_0)
        self.hidden_states = ["w0"] + hidden_states
        # first visible symbol is actually v0
        self.visible_symbols = ["v0"] + visible_symbols
    
        self.count_hidden = len(hidden_states) + 1
        self.count_visible = len(visible_symbols) + 1
    
        # transition matrix
        self.a = np.random.random(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        # emissions matrix
        self.b = np.random.random(self.count_hidden * self.count_visible).reshape(self.count_hidden, self.count_visible)
        
        # alpha matrix
        #self.alpha_matrix = np.zeros(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        # beta matrix
        #self._beta = np.zeros(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        
        self.a[0, 0] = 1
        for i in range(1, self.count_hidden):
            self.a[0, i] = 0
            
        self.b[0, 0] = 1
        for k in range(1, self.count_visible):
            self.b[0, k] = 0
        
        for j in range(1, self.count_hidden):
            self.b[j, 0] = 0
    
    def alpha(self, j, t, sequence, initial_hidden_state = 1, force_refresh = False):
        sequence = ["v0"] + sequence
        if force_refresh:
            self.alpha_matrix = np.zeros(self.count_hidden * len(sequence)).reshape(self.count_hidden,  len(sequence))
            #display("Force refresh")
        else:
            #display("getting precalculated value")
            return self.alpha_matrix[j, t]
        
        for t in range(len(sequence)):
            k = self.visible_symbols.index(sequence[t])
            for j in range(self.count_hidden):
                if t == 0 and j != initial_hidden_state:
                    self.alpha_matrix[j, t] = 0
                elif t == 0 and j == initial_hidden_state:
                    self.alpha_matrix[j, t] = 1
                else:
                    #display('j:{} t:{} k:{}, vk:{}'.format(j, t, k, sequence[t]))
                    suma = 0
                    for i in range(self.count_hidden):
                        #display('alpha_matrix_i{}_t{}:{} ai{}j{}:{}'.format(i, t-1, self.alpha_matrix[i, t-1], i, j, self.a[i,j]))
                        suma += self.alpha_matrix[i, t-1] * self.a[i, j]
                    
                    #display("bj{}k{}:{} result:{}".format(j, k,self.b[j,k], suma))
                    self.alpha_matrix[j, t] =  suma * self.b[j, k]
                    
    def beta(self, i, t, sequence, force_refresh = False):
        if force_refresh:
            self.beta_matrix = np.zeros(self.count_hidden * len(sequence)).reshape(self.count_hidden,  len(sequence))
            #display("Force refresh")
        else:
            #display("getting precalculated value")
            return self.beta_matrix[j, t]
        for t in reversed(range(len(sequence))):
            display("-")
            for j in range(self.count_hidden):
                if t == len(sequence) - 1 and j == 0:
                    self.beta_matrix[j, t] = 1
                elif t == len(sequence) - 1 and j != 0:
                    self.beta_matrix[j, t] = 0
                else:
                    k = self.visible_symbols.index(sequence[t])
                    suma = 0
                    for i in range(self.count_hidden):
                        display("beta_j{}_t{}:{} ai{}j{}:{}".format(i, t+1, self.beta_matrix[i, t + 1], i, j, self.a[i, j]))
                        suma += self.beta_matrix[i, t + 1] * self.a[j, i]

                    self.beta_matrix[j, t] = suma * self.b[j, k]
                    display("j:{} t:{} k:{} symbol:{} bj{}k{}:{} suma:{} result:{}".format(j, t, k, sequence[t],j, k, self.b[j, k], suma, self.beta_matrix[j, t]))
    
    def gamma(i, j, t, sequence):
        k = self.visible_symbols.index(sequence[t])
        return (self.alpha(i, t-1, sequence) * self.a[i,j] * self.b[j, k] * self.beta(j, t, sequence)) / self.alpha(0, len(sequence), sequence)

    def refined_a(i, j, sequence):
        expected_nr_of_transitions = sum([gamma(i, j, t, sequence) for t in range(len(sequence))])
        total_expected_nr_of_transitions = 1 #[gamma(i, k, t, sequence) for k in range]
        return expected_nr_of_transitions / total_expected_nr_of_transitions
    
    def refined_b(j, k, sequence):
        pass
    
    def learn(sequence):
        self.a = np.random.random(self.count_hidden ** 2).reshape(self.count_hidden, self.count_hidden)
        self.b = np.random.random(self.count_hidden * self.count_visible).reshape(self.count_hidden, self.count_visible)
        gain_in_accuracy = 1
        while gain_in_accuracy > 0.1:
            self.alpha(0, len(sequence) , sequence, force_refresh = True)
            self.beta(1, 0, sequence, force_refresh = True)
            new_aij = self.refined_a(i, j, sequence)
            new_bjk = self.refined_b(j, k, sequence)
            gain_in_accuracy = abs(a[i, j] - new_aij) + abs(b[j, k] - new_bjk)
            self.a[i,j] = new_aij
            sekf,b[j,k] = new_bjk

            


#model = HMM(["w1", "w2", "w3"], ["v1", "v2", "v3", "v4"])

#model.alpha(0, 4 , ["v3", "v1", "v2", "v1", "v0"], force_refresh = True)
#model.alpha_matrix

In [136]:
model3 = HMM(["w1", "w2", "w3"], ["v1", "v2", "v3", "v4"])


#                 W0   W1   W2    W3
aij = np.array([(  1,   0,   0,   0),  # W0
                (0.2, 0.3, 0.1, 0.4),  # W1
                (0.2, 0.5, 0.2, 0.1),  # W2
                (0.7, 0.1, 0.1, 0.1)]) # W3


#                 V0   V1   V2   V3   V4
bjk = np.array([(  1,   0,   0,   0,   0),   # W0
                (  0, 0.3, 0.4, 0.1, 0.2),   # W1
                (  0, 0.1, 0.1, 0.7, 0.1),   # W2
                (  0, 0.5, 0.2, 0.1, 0.2)])  # W3

model3.a = aij
model3.b = bjk

model3.beta(1, 0, ["v1", "v3", "v2", "v0"], force_refresh = True)
model3.beta_matrix

'-'

'-'

'beta_j0_t3:1.0 ai0j0:1.0'

'beta_j1_t3:0.0 ai1j0:0.2'

'beta_j2_t3:0.0 ai2j0:0.2'

...

'beta_j1_t1:0.0082 ai1j3:0.4'

'beta_j2_t1:0.0406 ai2j3:0.1'

'beta_j3_t1:0.0024 ai3j3:0.1'

'j:3 t:0 k:1 symbol:v1 bj3k1:0.5 suma:0.00512 result:0.00256'

array([[0.      , 0.      , 0.      , 1.      ],
       [0.002244, 0.0082  , 0.08    , 0.      ],
       [0.001246, 0.0406  , 0.02    , 0.      ],
       [0.00256 , 0.0024  , 0.14    , 0.      ]])

In [137]:
model2 = HMM(["w1", "w2", "w3"], ["v1", "v2", "v3", "v4"])


#                 W0   W1   W2    W3
aij = np.array([(  1,   0,   0,   0),  # W0
                (0.2, 0.3, 0.1, 0.4),  # W1
                (0.2, 0.5, 0.2, 0.1),  # W2
                (0.7, 0.1, 0.1, 0.1)]) # W3


#                 V0   V1   V2   V3   V4
bjk = np.array([(  1,   0,   0,   0,   0),   # W0
                (  0, 0.3, 0.4, 0.1, 0.2),   # W1
                (  0, 0.1, 0.1, 0.7, 0.1),   # W2
                (  0, 0.5, 0.2, 0.1, 0.2)])  # W3

model2.a = aij
model2.b = bjk

model2.alpha(0, 4 , ["v1", "v3", "v2", "v0"], force_refresh = True)
model2.alpha_matrix

array([[0.       , 0.       , 0.       , 0.       , 0.0018218],
       [1.       , 0.09     , 0.0052   , 0.005192 , 0.       ],
       [0.       , 0.01     , 0.0217   , 0.000543 , 0.       ],
       [0.       , 0.2      , 0.0057   , 0.000964 , 0.       ]])

In [138]:
model2.a

array([[1. , 0. , 0. , 0. ],
       [0.2, 0.3, 0.1, 0.4],
       [0.2, 0.5, 0.2, 0.1],
       [0.7, 0.1, 0.1, 0.1]])

In [139]:
model2.b

array([[1. , 0. , 0. , 0. , 0. ],
       [0. , 0.3, 0.4, 0.1, 0.2],
       [0. , 0.1, 0.1, 0.7, 0.1],
       [0. , 0.5, 0.2, 0.1, 0.2]])